In [1]:
import json
from osgeo import ogr
from shapely.geometry import mapping, shape
import xml.etree.ElementTree as ET

ogr.UseExceptions()

raw = [
    p
    for p in json.load(open("raw/properties.geojson"))["features"]
    if p["properties"]["houseno"] != 0  # property does not have an address
]

In [2]:
# merge properties that have the same address

# there are some properties that have seperate polygons in the same geometry,
# I assume these were purchased seperately, but with the same address?

addresses = {}
for p in raw:
    a = p["properties"]["houseaddress"]
    if a in addresses:
        addresses[a]["geometry"] = mapping(
            shape(addresses[a]["geometry"]).union(shape(p["geometry"]))
        )
        addresses[a]["properties"]["propertyno"] = (
            str(addresses[a]["properties"]["propertyno"])
            + ";"
            + str(p["properties"]["propertyno"])
        )
    else:
        addresses[a] = p

In [3]:
# convert and export

inconsistent = []
xml_id = -1
root = ET.Element("osm", version="0.6", generator="JOSM")

for a, p in addresses.items():
    ps = p["properties"]

    # generate tags
    tags = {
        "addr:city": "Townsville",
        "addr:housenumber": str(ps["houseno"]) + ps["housenosuffix"],
        "addr:postcode": ps["postcode"],
        "addr:street": ps["streetname"],
        "addr:suburb": ps["locality"].title(),
        "source": "Townsville City Council",
    }
    if ps["housenoto"] != 0:
        tags["addr:housenumber"] += "-" + str(ps["housenoto"]) + ps["housenotosuffix"]
    if ps["unitno"] != 0:
        tags["addr:unit"] = str(ps["unitno"]) + ps["unitnosuffix"]
        if ps["unitnoto"] != 0:
            tags["addr:unit"] += "-" + str(ps["unitnoto"]) + ps["unitnotosuffix"]

    # check for inconsistencies
    addr = f"{tags['addr:housenumber']} {tags['addr:street']} {tags['addr:suburb'].upper()} QLD {tags['addr:postcode']}"
    if "addr:unit" in tags:
        addr = tags["addr:unit"] + "/" + addr
    try:
        assert addr == ps["houseaddress"]
    except AssertionError as e:
        inconsistent.append((addr,ps["houseaddress"]))

    # generate XML for JOSM
    point = (
        ogr.CreateGeometryFromJson(json.dumps(p["geometry"]))
        .PointOnSurface()
        .GetPoint()
    )
    node = ET.SubElement(
        root, "node", id=str(xml_id), lat=str(point[1]), lon=str(point[0])
    )
    xml_id -= 1
    for k, v in tags.items():
        ET.SubElement(node, "tag", k=k, v=str(v))


ET.ElementTree(root).write("osm/addresses.osm")
print(len(inconsistent), "addresses are inconsistent:")
for a in inconsistent:
    print("    gen:", a[0])
    print("    raw:", a[1])
    print()

12 addresses are inconsistent:
    gen: 9A-11A Nora Road BLACK RIVER QLD 4818
    raw: 9-11 Nora Road BLACK RIVER QLD 4818

    gen: 106A-107 The Strand NORTH WARD QLD 4810
    raw: 106-107 The Strand NORTH WARD QLD 4810

    gen: 843B-855 Ingham Road BOHLE QLD 4818
    raw: 843-855 Ingham Road BOHLE QLD 4818

    gen: 857A-863 Riverway Drive CONDON QLD 4815
    raw: 857-863 Riverway Drive CONDON QLD 4815

    gen: 21A-33A Isley Street NORTH WARD QLD 4810
    raw: 21-33 Isley Street NORTH WARD QLD 4810

    gen: 18A-18B Leopold Street AITKENVALE QLD 4814
    raw: 18-18 Leopold Street AITKENVALE QLD 4814

    gen: 5-5A Melton Terrace TOWNSVILLE CITY QLD 4810
    raw: 5-5 Melton Terrace TOWNSVILLE CITY QLD 4810

    gen: 283A-283B Ingham Road GARBUTT QLD 4814
    raw: 283-283 Ingham Road GARBUTT QLD 4814

    gen: 17A-19A Whyte Street HERMIT PARK QLD 4812
    raw: 17-19 Whyte Street HERMIT PARK QLD 4812

    gen: 12A-14 MacIntosh Street HERMIT PARK QLD 4812
    raw: 12-14 MacIntosh Stree